In [14]:
!wget -P ../data/raw 123.57.42.89/Dataset_ict/LogoDet-3K.zip

--2025-02-04 06:26:41--  http://123.57.42.89/Dataset_ict/LogoDet-3K.zip
Connecting to 123.57.42.89:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3079016292 (2.9G) [application/zip]
Saving to: ‘../data/raw/LogoDet-3K.zip’

LogoDet-3K.zip        0%[                    ]  21.24M   645KB/s    eta 92m 38s^C


In [ ]:
!unzip -qq ../data/raw/LogoDet-3K.zip -d ../data/raw

In [ ]:
!wget -P ../models https://github.com/toxamontsg/yolo_models/raw/refs/heads/main/yolo5s_logo.pt

In [1]:
import sys
import os
import random

# Добавьте путь к вашему проекту в sys.path
sys.path.append('..')

from src.data_processing import DataSplit
from src.model import FewShotLogoRecognizer
from src.dataset import FewShotLogoDataset
from src.utils import augment_image

random.seed(42)

In [17]:
data_split = DataSplit(dataset_type='LogoDet-3K')
data_split.load_data(
    input_path='../data/raw/LogoDet-3K',
    output_path='../data/processed'
)

In [2]:
dataset = FewShotLogoDataset(
    images_dir="../data/processed/images/val",
    annotations_dir="../data/processed/annotations/val",
    support_samples=5
)

In [3]:
support_set = dataset.get_support_set()
query_set = dataset.get_query_set()

brand_examples = {}
for img_path, brand in support_set:
    if brand not in brand_examples:
        brand_examples[brand] = []
    brand_examples[brand].append(img_path)

Если убрать detector_path то модель изпользует данные из датасета для детекции bbox

In [4]:
recognizer = FewShotLogoRecognizer(
    brand_examples=brand_examples,
    model_name="ViT-B/32",
    detector_path="../models/yolo5s_logo.pt"
    
)

Load brands: 100%|██████████| 2717/2717 [05:27<00:00,  8.31it/s]


In [5]:
def get_rand_example(dataset, include_antiexample=False):
    brands = list(dataset.brand_groups.keys())
    random_brand = random.choice(brands)
    imgs = dataset.brand_groups[random_brand]
    random_img = random.choice(imgs)

    if include_antiexample:
        # Выбираем другой бренд, отличный от random_brand
        other_brands = [brand for brand in brands if brand != random_brand]
        random_other_brand = random.choice(other_brands)
        other_imgs = dataset.brand_groups[random_other_brand]
        random_other_img = random.choice(other_imgs)

        return random_brand, random_other_img
    else:
        return random_brand, random_img

Коэффициент (coefficient):

В случае сигмоидной функции коэффициент (например, -10) определяет чувствительность функции. Большие значения делают функцию более чувствительной к изменениям расстояния.
Эмпирически подберите коэффициент, чтобы вероятность корректно отражала сходство.
Порог (threshold):

Порог (например, 0.6) сдвигает сигмоидную функцию. Это позволяет вам настроить, при каком значении расстояния вероятность будет равна 0.5.
Эмпирически подберите порог, чтобы вероятность корректно отражала сходство.

In [62]:
threshold = 0.6
coefficient = 10

In [85]:
brand , img_path = get_rand_example(dataset)
recognizer.detect_logos(img_path, brand, threshold=threshold, coefficient = coefficient )

[{'brand': 'fox river mills',
  'confidence': 0.8643677084417293,
  'bbox': (184, 157, 438, 219)},
 {'brand': 'fox river mills',
  'confidence': 0.8484706618201715,
  'bbox': (50, 130, 179, 232)}]

Параметры: 

augmet=True симулирует размытость и наклон

include_antiexample=False пример положительный
include_antiexample=True пример отрицательный

In [95]:
brand , img_path = get_rand_example(dataset)
recognizer.detect_logos(img_path, brand, augmet=True, threshold=threshold, coefficient = coefficient )

[{'brand': 'Ecco',
  'confidence': 0.8963421435183988,
  'bbox': (42, 110, 453, 206)}]

In [96]:
brand , img_path = get_rand_example(dataset, include_antiexample=True)
recognizer.detect_logos(img_path, brand, augmet=True, threshold=threshold, coefficient = coefficient )

[{'brand': 'hutchinson tires',
  'confidence': 0.7174080141650893,
  'bbox': (81, 112, 266, 191)}]

In [101]:
brand , img_path = get_rand_example(dataset, include_antiexample=True)
recognizer.detect_logos(img_path, brand, augmet=True, threshold=threshold, coefficient = coefficient )

[{'brand': 'other',
  'confidence': 0.3473157026794199,
  'bbox': (130, 164, 345, 203)}]